***Regressão Linear***

Treinar o modelo usando o knn:
    - fazer a regressão linear 
    - árvores de decisão 
    - Random Forest e variantes (e.g., XGBoost)
    - Support Vector Machines (SVM)
    - Redes Neuronais

features (x) - colunas de 0 a 11 
target (price, y) - coluna 12 

In [ ]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from category_encoders import TargetEncoder as te
from sklearn.preprocessing import OneHotEncoder as ohe




In [148]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [149]:
### ----------------TRAIN------------------------------------------------------
train['horsepower'] = train['engine'].str.extract('(\d+\.?\d*)HP').astype(float)
train['displacement'] = train['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)
train['cylinders'] = train['engine'].str.extract(r'(\d+)\s*Cylinder').astype(float)
train['fuel_type'] = train['fuel_type'].replace('not supported', 'unknown')

no_engine = (train['horsepower'].isnull() & train['horsepower'].isnull() & train['horsepower'].isnull())
train['eletric'] = np.where(no_engine, 1,0)
train.drop(columns= ['engine'], inplace=True)

### ----------------TESTE------------------------------------------------------
test['horsepower'] = test['engine'].str.extract('(\d+\.?\d*)HP').astype(float)
test['displacement'] = test['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)
test['cylinders'] = test['engine'].str.extract(r'(\d+)\s*Cylinder').astype(float)
test['fuel_type'] = test['fuel_type'].replace('not supported', 'unknown')

no_engine = (test['horsepower'].isnull() & test['displacement'].isnull() & test['cylinders'].isnull())
test['eletric'] = np.where(no_engine, 1, 0)
test.drop(columns=['engine'], inplace=True)

In [150]:
### ----------------TRAIN------------------------------------------------------

train = train.fillna({'fuel_type': 'unknown', 'clean_title': 'unknown', 'accident': 'unknown', 'horsepower': 'unknown', 'displacement': 'unknown', 'cylinders': 'unknown'})
replace_cols = ['fuel_type', 'ext_col', 'int_col']

for col in replace_cols:
    train[col] = train[col].replace({'–': 'unknown'})
    
### ----------------TEST------------------------------------------------------

test = test.fillna({'fuel_type': 'unknown', 'clean_title': 'unknown', 'accident': 'unknown', 'horsepower': 'unknown', 'displacement': 'unknown', 'cylinders': 'unknown'})
    
for col in ['fuel_type', 'ext_col', 'int_col']:
    test[col] = test[col].replace({'–': 'unknown'})



In [151]:
accident = {
    'None reported': 0,
    'At least 1 accident or damage reported' : 1,
    'unknown' : 0
}

### ----------------TRAIN------------------------------------------------------
train['clean_title'] = train['clean_title'].astype('category')
train['clean_title'] = train['clean_title'].cat.codes

train['accident'] = train['accident'].map(accident)

### ----------------TEST------------------------------------------------------

test['clean_title'] = test['clean_title'].astype('category')
test['clean_title'] = test['clean_title'].cat.codes

test['accident'] = test['accident'].map(accident)



In [152]:
ohe_fuel = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

### ----------------TRAIN------------------------------------------------------
train['fuel_type'] = train['fuel_type'].astype('category')
fuel_ohe_train = ohe_fuel.fit_transform(train[['fuel_type']])
fuel_ohe_train_df = pd.DataFrame(fuel_ohe_train, 
                                 columns=ohe_fuel.get_feature_names_out(['fuel_type']),
                                 index=train.index)

train = pd.concat([train.drop(columns=['fuel_type']), fuel_ohe_train_df], axis=1)

### ----------------TEST------------------------------------------------------
test['fuel_type'] = test['fuel_type'].astype('category')
fuel_ohe_test = ohe_fuel.transform(test[['fuel_type']])
fuel_ohe_test_df = pd.DataFrame(fuel_ohe_test, 
                                columns=ohe_fuel.get_feature_names_out(['fuel_type']),
                                index=test.index)

test = pd.concat([test.drop(columns=['fuel_type']), fuel_ohe_test_df], axis=1)

In [153]:

train['transmission_type'] = train['transmission'].str.extract(
    r'(A/T|M/T|CVT|Auto-Shift|Dual Shift|Overdrive Switch|Automatic|Manual)'
)
train['transmission_speed'] = train['transmission'].str.extract('(\d+\.?\d*)-Speed').astype(float)
train = train.fillna({'transmission_speed': 'unknown', 'transmission_type': 'unknown'})

train = train.drop('transmission', axis = 1)
 
# Extrair tipo de transmissão
test['transmission_type'] = test['transmission'].str.extract(
    r'(A/T|M/T|CVT|Auto-Shift|Dual Shift|Overdrive Switch|Automatic|Manual)'
)

# Extrair velocidade (pode não existir → vira NaN e depois tratamos)
test['transmission_speed'] = test['transmission'].str.extract('(\d+\.?\d*)-Speed')

# Converter transmission_speed para float (com errors='coerce' para evitar crash)
test['transmission_speed'] = pd.to_numeric(test['transmission_speed'], errors='coerce')

# Preencher valores em falta da mesma forma que no train
test = test.fillna({
    'transmission_speed': 'unknown',
    'transmission_type': 'unknown'
})

# Remover coluna original (igual ao train)
test = test.drop(columns=['transmission'])


In [154]:
train['transmission_type'] = train['transmission_type'].astype('category')
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output= False).set_output(transform='pandas')
ohetransform = ohe.fit_transform(train[['transmission_type']]).astype(int)
train = pd.concat([train, ohetransform], axis=1).drop(columns=['transmission_type'])

# Garantir que a coluna é category
test['transmission_type'] = test['transmission_type'].astype('category')

# Usar o encoder treinado no train (ohe)
ohe_test = ohe.transform(test[['transmission_type']]).astype(int)

# Criar DataFrame com as colunas corretas
ohe_test_df = pd.DataFrame(
    ohe_test,
    columns=ohe.get_feature_names_out(['transmission_type']),
    index=test.index
)

# Concatenar e remover a coluna original
test = pd.concat([test.drop(columns=['transmission_type']), ohe_test_df], axis=1)
train

,id,brand,model,model_year,milage,ext_col,int_col,accident,clean_title,price,...,fuel_type_unknown,transmission_speed,transmission_type_A/T,transmission_type_Automatic,transmission_type_CVT,transmission_type_Dual Shift,transmission_type_M/T,transmission_type_Manual,transmission_type_Overdrive Switch,transmission_type_unknown
0,0,MINI,Cooper S Base,2007,213000,Yellow,Gray,0,0,4200,...,0.0,unknown,1,0,0,0,0,0,0,0
1,1,Lincoln,LS V8,2002,143250,Silver,Beige,1,0,4999,...,0.0,unknown,1,0,0,0,0,0,0,0
2,2,Chevrolet,Silverado 2500 LT,2002,136731,Blue,Gray,0,0,13900,...,0.0,unknown,1,0,0,0,0,0,0,0
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Black,Black,0,0,45000,...,0.0,unknown,0,0,0,1,0,0,0,0
4,4,Mercedes-Benz,Metris Base,2021,7388,Black,Beige,0,0,97500,...,0.0,7.0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,White,Beige,0,0,27500,...,0.0,unknown,0,0,0,1,0,0,0,0
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,White,Black,1,0,30000,...,0.0,8.0,1,0,0,0,0,0,0,0
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,White,Black,0,0,86900,...,0.0,7.0,1,0,0,0,0,0,0,0
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Daytona Gray Pearl Effect,Black,0,1,84900,...,0.0,1.0,0,1,0,0,0,0,0,0


In [155]:
import re
ext_colors = train['ext_col'].astype(str)
int_colors = train['int_col'].astype(str)
ext_colors_test = test['ext_col'].astype(str)
int_colors_test = test['int_col'].astype(str)

one_ext_color = ext_colors[ext_colors.str.contains(r'^[A-Za-zÀ-ÿ]+$', regex=True)].unique().tolist()
one_int_color = int_colors[int_colors.str.contains(r'^[A-Za-zÀ-ÿ]+$', regex=True)].unique().tolist()

colors_map = {c.lower(): c for c in one_ext_color}
def normalize_color(color):
    lower_color = color.lower()
    for word, base_color in colors_map.items():
        if re.search(rf'\b{word}\b', lower_color):
            return base_color
    return color

In [156]:
train['ext_col']= ext_colors.apply(normalize_color)
train['int_col'] = int_colors.apply(normalize_color)

test['ext_col'] = ext_colors_test.apply(normalize_color)
test['int_col'] = int_colors_test.apply(normalize_color)

train['all_colors'] = train[['int_col', 'ext_col']].values.tolist()
test['all_colors'] = test[['int_col', 'ext_col']].values.tolist()


exploded_train = train['all_colors'].explode()
exploded_test = test['all_colors'].explode()


count_df_train = pd.crosstab(exploded_train.index, exploded_train)

count_df_test = pd.crosstab(exploded_test.index, exploded_test)

count_df_train_columns = [f"color_{c}" for c in count_df_train.columns]

count_df_test.columns = [f"color_{c}" for c in count_df_test.columns]


train = pd.concat([train, count_df_train], axis=1).drop(columns=['int_col', 'ext_col', 'all_colors'])
test = pd.concat([test.drop(columns=['int_col','ext_col','all_colors']), count_df_test], axis=1)

test

,id,brand,model,model_year,milage,accident,clean_title,horsepower,displacement,cylinders,...,color_Tension,color_Tupelo,color_Verde,color_Very Light Cashmere,color_Walnut,color_White,color_Wind Chill Pearl,color_Yellow,color_Yulong,color_unknown
0,188533,Land,Rover LR2 Base,2015,98000,0,0,240.0,2.0,4.0,...,0,0,0,0,0,1,0,0,0,0
1,188534,Land,Rover Defender SE,2020,9142,0,0,395.0,3.0,6.0,...,0,0,0,0,0,0,0,0,0,0
2,188535,Ford,Expedition Limited,2022,28121,0,1,unknown,3.5,unknown,...,0,0,0,0,0,1,0,0,0,0
3,188536,Audi,A6 2.0T Sport,2016,61258,0,1,unknown,unknown,unknown,...,0,0,0,0,0,0,0,1,0,0
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,0,0,252.0,2.0,4.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125685,314218,Mercedes-Benz,GL-Class GL 450 4MATIC,2014,83315,0,0,362.0,3.0,6.0,...,0,0,0,0,0,0,0,0,0,0
125686,314219,Audi,Q7 55 Prestige,2019,29336,0,1,unknown,unknown,unknown,...,0,0,0,0,0,1,0,0,0,0
125687,314220,Audi,A6 3.0T Premium Plus,2012,77634,0,0,333.0,3.0,6.0,...,0,0,0,0,0,0,0,0,0,0
125688,314221,Audi,Q7 3.0T Premium,2012,112000,0,0,333.0,3.0,6.0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
train['brand'] = train['brand'].astype('category')

# criar e treinar o encoder
ohe_brand = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
brand_train = ohe_brand.fit_transform(train[['brand']]).astype(int)

# concatenar com o train e remover a coluna original
train = pd.concat([train.drop(columns=['brand']), brand_train], axis=1)


# usar o encoder treinado no train
brand_test = ohe_brand.transform(test[['brand']]).astype(int)

# criar DataFrame com colunas corretas
brand_test_df = pd.DataFrame(
    brand_test,
    columns=ohe_brand.get_feature_names_out(['brand']),
    index=test.index
)

# concatenar e remover a coluna original
test = pd.concat([test.drop(columns=['brand']), brand_test_df], axis=1)

train


,id,model,model_year,milage,accident,clean_title,price,horsepower,displacement,cylinders,...,brand_Saab,brand_Saturn,brand_Scion,brand_Subaru,brand_Suzuki,brand_Tesla,brand_Toyota,brand_Volkswagen,brand_Volvo,brand_smart
0,0,Cooper S Base,2007,213000,0,0,4200,172.0,1.6,4.0,...,0,0,0,0,0,0,0,0,0,0
1,1,LS V8,2002,143250,1,0,4999,252.0,3.9,8.0,...,0,0,0,0,0,0,0,0,0,0
2,2,Silverado 2500 LT,2002,136731,0,0,13900,320.0,5.3,8.0,...,0,0,0,0,0,0,0,0,0,0
3,3,G90 5.0 Ultimate,2017,19500,0,0,45000,420.0,5.0,8.0,...,0,0,0,0,0,0,0,0,0,0
4,4,Metris Base,2021,7388,0,0,97500,208.0,2.0,4.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Escalade ESV Platinum,2017,49000,0,0,27500,420.0,6.2,8.0,...,0,0,0,0,0,0,0,0,0,0
188529,188529,AMG C 43 AMG C 43 4MATIC,2018,28600,1,0,30000,385.0,3.0,6.0,...,0,0,0,0,0,0,0,0,0,0
188530,188530,AMG GLC 63 Base 4MATIC,2021,13650,0,0,86900,469.0,4.0,8.0,...,0,0,0,0,0,0,0,0,0,0
188531,188531,S5 3.0T Prestige,2022,13895,0,1,84900,unknown,3.0,unknown,...,0,0,0,0,0,0,0,0,0,0


In [158]:
train


,id,model,model_year,milage,accident,clean_title,price,horsepower,displacement,cylinders,...,brand_Saab,brand_Saturn,brand_Scion,brand_Subaru,brand_Suzuki,brand_Tesla,brand_Toyota,brand_Volkswagen,brand_Volvo,brand_smart
0,0,Cooper S Base,2007,213000,0,0,4200,172.0,1.6,4.0,...,0,0,0,0,0,0,0,0,0,0
1,1,LS V8,2002,143250,1,0,4999,252.0,3.9,8.0,...,0,0,0,0,0,0,0,0,0,0
2,2,Silverado 2500 LT,2002,136731,0,0,13900,320.0,5.3,8.0,...,0,0,0,0,0,0,0,0,0,0
3,3,G90 5.0 Ultimate,2017,19500,0,0,45000,420.0,5.0,8.0,...,0,0,0,0,0,0,0,0,0,0
4,4,Metris Base,2021,7388,0,0,97500,208.0,2.0,4.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Escalade ESV Platinum,2017,49000,0,0,27500,420.0,6.2,8.0,...,0,0,0,0,0,0,0,0,0,0
188529,188529,AMG C 43 AMG C 43 4MATIC,2018,28600,1,0,30000,385.0,3.0,6.0,...,0,0,0,0,0,0,0,0,0,0
188530,188530,AMG GLC 63 Base 4MATIC,2021,13650,0,0,86900,469.0,4.0,8.0,...,0,0,0,0,0,0,0,0,0,0
188531,188531,S5 3.0T Prestige,2022,13895,0,1,84900,unknown,3.0,unknown,...,0,0,0,0,0,0,0,0,0,0


In [159]:
test

,id,model,model_year,milage,accident,clean_title,horsepower,displacement,cylinders,eletric,...,brand_Saab,brand_Saturn,brand_Scion,brand_Subaru,brand_Suzuki,brand_Tesla,brand_Toyota,brand_Volkswagen,brand_Volvo,brand_smart
0,188533,Rover LR2 Base,2015,98000,0,0,240.0,2.0,4.0,0,...,0,0,0,0,0,0,0,0,0,0
1,188534,Rover Defender SE,2020,9142,0,0,395.0,3.0,6.0,0,...,0,0,0,0,0,0,0,0,0,0
2,188535,Expedition Limited,2022,28121,0,1,unknown,3.5,unknown,0,...,0,0,0,0,0,0,0,0,0,0
3,188536,A6 2.0T Sport,2016,61258,0,1,unknown,unknown,unknown,1,...,0,0,0,0,0,0,0,0,0,0
4,188537,A6 2.0T Premium Plus,2018,59000,0,0,252.0,2.0,4.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125685,314218,GL-Class GL 450 4MATIC,2014,83315,0,0,362.0,3.0,6.0,0,...,0,0,0,0,0,0,0,0,0,0
125686,314219,Q7 55 Prestige,2019,29336,0,1,unknown,unknown,unknown,1,...,0,0,0,0,0,0,0,0,0,0
125687,314220,A6 3.0T Premium Plus,2012,77634,0,0,333.0,3.0,6.0,0,...,0,0,0,0,0,0,0,0,0,0
125688,314221,Q7 3.0T Premium,2012,112000,0,0,333.0,3.0,6.0,0,...,0,0,0,0,0,0,0,0,0,0
